## Dataset

In [1]:
import time
import random as rd
import csv

In [2]:
def crea_dataset():
    column_names=['server_id', 'application_id', 'start_timestamp']
    server_list = [x for x in range(1,11)]
    application_list = ['app'+ str(x)  for x in range(1,11)]
    start_time_list=[int(time.time())+x for x in range(1,1000)]
    data=[]
    data.append(column_names)
    for i in range(1000):
        raw=[i,rd.choice(server_list),rd.choice(application_list), rd.choice(start_time_list)]
        data.append(raw)
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return data    

In [3]:
crea_dataset();

## Analysis

In [4]:
import numpy as np
import pandas as pd

In [5]:
df= pd.DataFrame.from_csv('dataset.csv')

In [6]:
df.head()

,server_id,application_id,start_timestamp
0,8,app4,1508344483
1,6,app6,1508344610
2,2,app8,1508345115
3,5,app8,1508345186
4,8,app5,1508344661


In [7]:
df.columns

Index(['server_id', 'application_id', 'start_timestamp'], dtype='object')

In [8]:
df=df.sort_values(by=['start_timestamp'], ascending=[True])

In [9]:
df = df.reset_index(drop=True)
df[:5]

,server_id,application_id,start_timestamp
0,10,app6,1508344284
1,10,app1,1508344284
2,8,app2,1508344286
3,10,app1,1508344287
4,5,app7,1508344288


In [10]:
df.groupby(df['server_id']).get_group(2).describe()

,server_id,start_timestamp
count,108.0,1.080000e+02
mean,2.0,1.508345e+09
std,0.0,2.985240e+02
min,2.0,1.508344e+09
25%,2.0,1.508345e+09
50%,2.0,1.508345e+09
75%,2.0,1.508345e+09
max,2.0,1.508345e+09


In [11]:
p=pd.Series()
for i in range(1,11):    
    s=df.groupby(['server_id'])['start_timestamp'].get_group(i).diff().shift(-1)
    p=p.append(s)
p.sort_index()
df['app_time']=p

In [12]:
df['app_time'].describe()

count    990.000000
mean       9.845455
std        9.827293
min        0.000000
25%        3.000000
50%        7.000000
75%       14.000000
max       69.000000
Name: app_time, dtype: float64

In [13]:
df['app_time'].isnull().sum()

10

In [14]:
df_filtered=df[(df.app_time > 45 ) & (df.app_time < 300)]
df_filtered

,server_id,application_id,start_timestamp,app_time
5,8,app1,1508344289,51.0
145,1,app1,1508344421,51.0
153,8,app4,1508344428,50.0
184,9,app5,1508344467,69.0
323,5,app5,1508344604,48.0
356,7,app10,1508344636,49.0
368,5,app1,1508344652,52.0
531,8,app6,1508344811,65.0


In [15]:
df['application_id'].groupby(df['server_id']).describe()

,count,unique,top,freq
server_id,,,,
1,106,10,app9,18
2,108,10,app2,17
3,112,10,app8,17
4,89,10,app8,12
5,99,10,app1,15
6,98,10,app9,13
7,108,10,app2,13
8,84,10,app3,15
9,83,10,app10,10


In [44]:
df2=df.groupby(['server_id', 'application_id'])['app_time'].describe(percentiles=[.1])
df2;

In [61]:
df2.columns

Index(['count', 'mean', 'std', 'min', '10%', '50%', 'max'], dtype='object')

In [66]:
REF=pd.DataFrame(df2['10%'])

In [71]:
REF.xs(1, level='server_id')

,10%
application_id,
app1,5.5
app10,1.6
app2,1.2
app3,1.6
app4,1.3
app5,2.2
app6,1.7
app7,2.3
app8,2.6
